In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/FDS/processed.csv')
df.head()


,title,content,publisher,published_date,combined
0,intel is back with a beat and raise,"for years, intel’s r&d had stagnated, as spend...",Seeking Alpha,2021-04-27 09:35:45,intel back beat raise year intel ’ rd stagnate...
1,"tech time: microsoft, advanced micro devices, ...","ready or not, it’s time for all of us to put o...",Benzinga,2021-04-27 14:11:40,tech time microsoft advanced micro device alph...
2,"fed, big tech vie for attention ahead of micro...",NaN,Investing.com,2021-04-27 14:51:00,fed big tech vie attention ahead microsoft alp...
3,6 causes of the global semiconductor shortage,the world's top chipmakers still can't manufac...,The Motley Fool,2021-04-27 16:28:35,cause global semiconductor shortage world top ...
4,tesla: full self-driving success not guarantee...,tesla claims that it will be able to reach ful...,Seeking Alpha,2021-04-28 07:38:12,tesla full selfdriving success guaranteed yet ...


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import torch

# Load FinBERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.eval()  # Set model to evaluation mode

# Initialize the sentiment analysis pipeline
# Initialize the sentiment analysis pipeline with truncation enabled
nlp = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0,
    truncation=True,  # Truncate long sequences
    max_length=512  # Set the max length to 512 tokens
)
# Test sentence
sentence = "cathie wood add share shopify sell square pile coinbase cathie woodled ark investment management monday snapped share canadian ecommerce company shopify inc nyse shop worth million stock jumped ahead scheduled earnings report later week investment firm ’ ark innovation etf nyse arkk bought share company closed higher monday investment firm also sold share digital payment company square inc nyse sq estimated worth million monday ’ close shopify caters small midsize business firm two segment — subscription solution fiscal revenue merchant solution fiscal revenue see also shopifys stock trading higher today arkk sold share square full story available benzingacom"

# Get the sentiment prediction
result = nlp(sentence)

# Print the result
print(f"Sentiment: {result[0]['label']}")
print(f"Confidence Score: {result[0]['score']}")

print(f'Using device: {device}')

Sentiment: neutral
Confidence Score: 0.7787325978279114
Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from tqdm.notebook import tqdm
from typing import Union, List

def get_sentiment_score(texts: Union[str, List[str]],
                       show_progress: bool = True) -> Union[float, List[float]]:
    # Handle single text input
    if isinstance(texts, str):
        result = nlp(texts)[0]
        sentiment = result['label']
        score = result['score']

        if sentiment == "positive":
            return score
        elif sentiment == "neutral":
            return 0
        else:  # Negative sentiment
            return -score

    # Handle list of texts
    scores = []
    iterator = tqdm(texts, desc="Analyzing sentiment") if show_progress else texts

    for text in iterator:
        result = nlp(text)[0]
        sentiment = result['label']
        score = result['score']

        if sentiment == "positive":
            scores.append(score)
        elif sentiment == "neutral":
            scores.append(0)
        else:  # Negative sentiment
            scores.append(-score)

    return scores


In [ ]:
df['combined'] = df['combined'].astype(str)

In [ ]:
df['sentiment_score'] = df['combined'].apply(get_sentiment_score)



print("Sentiment analysis completed. Results saved to 'sentiment_scores.csv'.")

Sentiment analysis completed. Results saved to 'sentiment_scores.csv'.


In [ ]:
df.to_csv('INTC_unseen_sentiment.csv', index=False)